In [1]:
import pandas as pd

In [2]:
df_msr = pd.read_csv('../MSR/MSR_data_cleaned.csv')

/Users/mahbubcseju/opt/anaconda3/envs/codebert/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (20,22,23,27,28,29) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
df_devign = pd.read_json('../devign/function.json')

In [4]:
df_msr_ = []
for index, row in df_msr.iterrows():
    df_msr_.append(row)

In [5]:
df_devign_ = []
for index, row in df_devign.iterrows():
    df_devign_.append(row)

In [6]:
df_msr_[0]

Unnamed: 0                                                                      0
Access Gained                                                                None
Attack Origin                                                              Remote
Authentication Required                                             Single system
Availability                                                              Partial
CVE ID                                                              CVE-2015-8467
CVE Page                            https://www.cvedetails.com/cve/CVE-2015-8467/
CWE ID                                                                    CWE-264
Complexity                                                                 Medium
Confidentiality                                                           Partial
Integrity                                                                 Partial
Known Exploits                                                                NaN
Publish Date    

In [7]:
df = pd.read_csv('balanced_0.1.csv')

In [8]:
df

,Unnamed: 0,dataset,example_index,project,commit_id,target,split
0,6,Devign,6,FFmpeg,afb4632cc30e83287338690c785ebac180436a59,1,train
1,18,Devign,18,qemu,3e305e4a4752f70c0b5c3cf5b43ec957881714f7,1,train
2,21,Devign,21,FFmpeg,323e6fead07c75f418e4b60704a4f437bb3483b2,1,train
3,30,Devign,30,FFmpeg,38477e1981a7729b02d9a2f45142c53dc78625fe,0,train
4,49,Devign,49,FFmpeg,9f61abc8111c7c43f49ca012e957a108b9cc7610,0,train
...,...,...,...,...,...,...,...
8303,215829,MSR,188593,Android,5a9753fca56f0eeb9f61e342b2fccffc364f9426,1,test
8304,215837,MSR,188601,Android,5a9753fca56f0eeb9f61e342b2fccffc364f9426,1,test
8305,215841,MSR,188605,Android,5a9753fca56f0eeb9f61e342b2fccffc364f9426,1,test
8306,215849,MSR,188613,Android,5a9753fca56f0eeb9f61e342b2fccffc364f9426,1,test


In [9]:
import json

In [10]:
def get_ind_data(row):
    if row['dataset'] == 'MSR':
        return json.dumps({
            'func': df_msr_[row['example_index']]['func_before'],
            'target': df_msr_[row['example_index']]['vul'],
            'dataset': row['dataset'],
            'idx': row['example_index'],
            'project': row['project']
        }) +"\n"
    elif row['dataset'] == 'Devign':
        return json.dumps({
            'func': df_devign_[row['example_index']]['func'],
            'target': df_devign_[row['example_index']]['target'],
            'dataset': row['dataset'],
            'idx': row['example_index'],
            'project': row['project']
        }) +"\n"
    assert False

In [11]:
def get_data(df):
    train = []
    test = []
    valid = []
    for ind, row in df.iterrows():
        if row['split'] == 'train':
            train.append(get_ind_data(row))
        if row['split'] == 'test':
            test.append(get_ind_data(row))
        if row['split'] == 'valid':
            valid.append(get_ind_data(row))
    return train, test, valid

In [12]:
def writejsonl(data, file_path):
    f = open(file_path, 'w')
    f.writelines(data)
    f.close()

In [15]:
import os
import shutil

In [16]:
fold_num = ['0.01', '0.05']
for dataset_name in [ 'imbalanced']:
    for num in fold_num:
        dataset = pd.read_csv('{}_{}.csv'.format(dataset_name, num))
        result = [[], [], []]
        result = get_data(dataset)
        folder = '{}_{}'.format(dataset_name, num)
        if not os.path.exists(folder):
            os.mkdir(folder)
        else:
            shutil.rmtree(folder)
            os.mkdir(folder)
        writejsonl(result[0], '{}/train.jsonl'.format(folder))
        writejsonl(result[1], '{}/test.jsonl'.format(folder))
        writejsonl(result[2], '{}/valid.jsonl'.format(folder))